## Conjugate Gradient Method

In [7]:
import numpy as np
import matplotlib.image as mpimg
from scipy.ndimage import gaussian_filter


def laplace_dirichlet(u: np.ndarray) -> np.ndarray:
    up = np.pad(u, ((1, 1), (1, 1)), mode="constant", constant_values=0)
    out = (
        -4.0 * up[1:-1, 1:-1]
        + up[0:-2, 1:-1]
        + up[2:, 1:-1]
        + up[1:-1, 0:-2]
        + up[1:-1, 2:]
    )
    return out

def M_matrix(u, v, Ix, Iy, lam):
    Mu = (Ix * Ix) * u + (Ix * Iy) * v - lam * laplace_dirichlet(u)
    Mv = (Ix * Iy) * u + (Iy * Iy) * v - lam * laplace_dirichlet(v)
    return Mu, Mv

def compute_derivatives(I0, I1):
    I0x = np.zeros_like(I0)
    I1x = np.zeros_like(I1)
    I0x[:, :-1] = I0[:, 1:] - I0[:, :-1]
    I0x[:, -1]  = I0[:, -1] - I0[:, -2]
    I1x[:, :-1] = I1[:, 1:] - I1[:, :-1]
    I1x[:, -1]  = I1[:, -1] - I1[:, -2]
    Ix = 0.5 * (I0x + I1x)

    I0y = np.zeros_like(I0)
    I1y = np.zeros_like(I1)
    I0y[:-1, :] = I0[1:, :] - I0[:-1, :]
    I0y[-1, :]  = I0[-1, :] - I0[-2, :]
    I1y[:-1, :] = I1[1:, :] - I1[:-1, :]
    I1y[-1, :]  = I1[-1, :] - I1[-2, :]
    Iy = 0.5 * (I0y + I1y)

    return Ix, Iy


def OF_cg(u0, v0, Ix, Iy, reg, rhsu, rhsv, tol=1e-8, maxit=2000, verbose=False):
    u, v = u0.copy(), v0.copy()
    Au, Av = M_matrix(u, v, Ix, Iy, reg)
    ru, rv = rhsu - Au, rhsv - Av

    def dot2(a1, a2, b1, b2):
        return float(np.sum(a1 * b1) + np.sum(a2 * b2))

    norm_r0 = np.sqrt(dot2(ru, rv, ru, rv))
    if norm_r0 == 0.0:
        return u, v, {"it": 0, "relres": 0.0, "converged": True}

    pu, pv = ru.copy(), rv.copy()
    relres, it = 1.0, 0

    for k in range(1, maxit + 1):
        Apu, Apv = M_matrix(pu, pv, Ix, Iy, reg)
        denom = dot2(pu, pv, Apu, Apv)
        rr = dot2(ru, rv, ru, rv)
        alpha = rr / denom
        u += alpha * pu
        v += alpha * pv
        ru -= alpha * Apu
        rv -= alpha * Apv

        relres = np.sqrt(dot2(ru, rv, ru, rv)) / norm_r0
        if verbose and (k % 10 == 0 or relres < tol):
            print(f"it={k:4d} relres={relres:.3e}")
        if relres < tol:
            it = k
            break

        rr_new = dot2(ru, rv, ru, rv)
        beta = rr_new / rr
        pu, pv = ru + beta * pu, rv + beta * pv
        it = k

    return u, v, {"it": it, "relres": relres, "converged": relres < tol}

def build_right_hand_side(Ix, Iy, It):
    return -Ix * It, -Iy * It

def run_optical_flow(frame0_path, frame1_path, lam=1.0, sigma=1.0, tol=1e-8, maxit=2000):
    I0 = mpimg.imread(frame0_path).astype(float)
    I1 = mpimg.imread(frame1_path).astype(float)
    if I0.ndim == 3:
        I0 = np.mean(I0, axis=2)
        I1 = np.mean(I1, axis=2)

    I0 = gaussian_filter(I0, sigma)
    I1 = gaussian_filter(I1, sigma)

    Ix,Iy = compute_derivatives(I0,I1)
    It = I1 - I0

    rhsu, rhsv = build_right_hand_side(Ix, Iy, It)
    u0, v0 = np.zeros_like(I0), np.zeros_like(I0)

    u, v, info = OF_cg(u0, v0, Ix, Iy, lam, rhsu, rhsv, tol, maxit, verbose=True)

    print(f"CG finished in {info['it']} iterations, relres={info['relres']:.2e}, converged={info['converged']}")
    return u, v, info


frame0_path = "/Users/ethanclement/Documents/NLAProject/frame10.png"
frame1_path = "/Users/ethanclement/Documents/NLAProject/frame11.png"

print(run_optical_flow(frame0_path ,frame1_path ))


it=  10 relres=1.436e+00
it=  20 relres=1.331e+00
it=  30 relres=1.210e+00
it=  40 relres=1.052e+00
it=  50 relres=8.778e-01
it=  60 relres=7.130e-01
it=  70 relres=5.967e-01
it=  80 relres=4.992e-01
it=  90 relres=4.154e-01
it= 100 relres=3.554e-01
it= 110 relres=3.082e-01
it= 120 relres=2.619e-01
it= 130 relres=2.096e-01
it= 140 relres=1.803e-01
it= 150 relres=1.507e-01
it= 160 relres=1.246e-01
it= 170 relres=9.923e-02
it= 180 relres=8.198e-02
it= 190 relres=7.041e-02
it= 200 relres=5.994e-02
it= 210 relres=5.189e-02
it= 220 relres=4.611e-02
it= 230 relres=3.973e-02
it= 240 relres=3.315e-02
it= 250 relres=2.804e-02
it= 260 relres=2.301e-02
it= 270 relres=1.887e-02
it= 280 relres=1.565e-02
it= 290 relres=1.289e-02
it= 300 relres=1.056e-02
it= 310 relres=8.685e-03
it= 320 relres=7.409e-03
it= 330 relres=6.280e-03
it= 340 relres=5.562e-03
it= 350 relres=4.698e-03
it= 360 relres=3.862e-03
it= 370 relres=3.240e-03
it= 380 relres=2.856e-03
it= 390 relres=2.458e-03
it= 400 relres=2.074e-03


In [ ]:
import numpy as np
import matplotlib.image as mpimg
from scipy.ndimage import gaussian_filter

# ============================================================
# Basic Laplacian & Operator (your original)
# ============================================================

def laplace_dirichlet(u: np.ndarray) -> np.ndarray:
    """5-point Laplacian with zero-padding outside the image (Dirichlet BC)."""
    up = np.pad(u, ((1, 1), (1, 1)), mode="constant", constant_values=0)
    out = (
        -4.0 * up[1:-1, 1:-1]
        + up[0:-2, 1:-1]
        + up[2:, 1:-1]
        + up[1:-1, 0:-2]
        + up[1:-1, 2:]
    )
    return out

def M_matrix(u, v, Ix, Iy, lam):
    """Fine-grid operator A(u,v) for your CG solver."""
    Mu = (Ix * Ix) * u + (Ix * Iy) * v - lam * laplace_dirichlet(u)
    Mv = (Ix * Iy) * u + (Iy * Iy) * v - lam * laplace_dirichlet(v)
    return Mu, Mv


def compute_derivatives(I0, I1):
    """Forward differences in x and y (averaged over the two frames)."""
    I0x = np.zeros_like(I0)
    I1x = np.zeros_like(I1)
    I0x[:, :-1] = I0[:, 1:] - I0[:, :-1]
    I0x[:, -1]  = I0[:, -1] - I0[:, -2]
    I1x[:, :-1] = I1[:, 1:] - I1[:, :-1]
    I1x[:, -1]  = I1[:, -1] - I1[:, -2]
    Ix = 0.5 * (I0x + I1x)

    I0y = np.zeros_like(I0)
    I1y = np.zeros_like(I1)
    I0y[:-1, :] = I0[1:, :] - I0[:-1, :]
    I0y[-1, :]  = I0[-1, :] - I0[-2, :]
    I1y[:-1, :] = I1[1:, :] - I1[:-1, :]
    I1y[-1, :]  = I1[-1, :] - I1[-2, :]
    Iy = 0.5 * (I0y + I1y)

    return Ix, Iy

def build_right_hand_side(Ix, Iy, It):
    """RHS from the Horn–Schunck equations: -Ix It, -Iy It."""
    return -Ix * It, -Iy * It

def OF_cg(u0, v0, Ix, Iy, reg, rhsu, rhsv, tol=1e-8, maxit=2000, verbose=False):
    """Standard CG for the optical flow system with your operator."""
    u, v = u0.copy(), v0.copy()
    Au, Av = M_matrix(u, v, Ix, Iy, reg)
    ru, rv = rhsu - Au, rhsv - Av

    def dot2(a1, a2, b1, b2):
        return float(np.sum(a1 * b1) + np.sum(a2 * b2))

    norm_r0 = np.sqrt(dot2(ru, rv, ru, rv))
    if norm_r0 == 0.0:
        return u, v, {"it": 0, "relres": 0.0, "converged": True}

    pu, pv = ru.copy(), rv.copy()
    relres, it = 1.0, 0

    for k in range(1, maxit + 1):
        Apu, Apv = M_matrix(pu, pv, Ix, Iy, reg)
        denom = dot2(pu, pv, Apu, Apv)
        rr = dot2(ru, rv, ru, rv)
        alpha = rr / denom
        u += alpha * pu
        v += alpha * pv
        ru -= alpha * Apu
        rv -= alpha * Apv

        relres = np.sqrt(dot2(ru, rv, ru, rv)) / norm_r0
        if verbose and (k % 10 == 0 or relres < tol):
            print(f"it={k:4d} relres={relres:.3e}")
        if relres < tol:
            it = k
            break

        rr_new = dot2(ru, rv, ru, rv)
        beta = rr_new / rr
        pu, pv = ru + beta * pu, rv + beta * pv
        it = k

    return u, v, {"it": it, "relres": relres, "converged": relres < tol}

def zero_boundary(a):
    """Enforce homogeneous Dirichlet boundary conditions."""
    a = a.copy()
    a[0, :]  = 0.0
    a[-1, :] = 0.0
    a[:, 0]  = 0.0
    a[:, -1] = 0.0
    return a

def laplacian(u):
    """
    5-point Laplacian on interior points only (no 1/h^2 factor),
    boundaries left untouched (assumed to be Dirichlet = 0 in solution).
    """
    n, m = u.shape
    L = np.zeros_like(u)
    L[1:n-1, 1:m-1] = (
        u[0:n-2, 1:m-1] + u[2:n, 1:m-1] +
        u[1:n-1, 0:m-2] + u[1:n-1, 2:m] -
        4.0 * u[1:n-1, 1:m-1]
    )
    return L

def A(u, v, Ix, Iy, reg, level=0):
    """
    Multigrid version of A(u,v):

    A(u) = Ix^2 u + Ix Iy v - reg * (1/h^2) * Laplacian(u)
    with h = 2^level  ->  1/h^2 = 4^(-level)
    """
    u = zero_boundary(u)
    v = zero_boundary(v)

    h2inv = 4.0 ** (-level)  # 1/h^2 for grid spacing h = 2^level

    Lu = laplacian(u)
    Lv = laplacian(v)

    Ix2  = Ix * Ix
    Iy2  = Iy * Iy
    IxIy = Ix * Iy

    Au = Ix2 * u + IxIy * v - reg * (h2inv * Lu)
    Av = IxIy * u + Iy2 * v - reg * (h2inv * Lv)

    return zero_boundary(Au), zero_boundary(Av)

def smoothing(u0, v0, Ix, Iy, reg, rhsu, rhsv, s1, level=0, parity=0):
    """
    Red–black Gauss–Seidel smoother with local 2×2 solves on each pixel.
    - u0, v0: current approximation (full grid, including boundary)
    - Ix, Iy: image derivatives on this level
    - reg: lambda
    - rhsu, rhsv: right-hand side at this level
    - s1: number of sweeps
    - level: grid level (0=fine)
    - parity: 0 -> red-black, 1 -> black-red (for symmetric GS)
    """
    u = u0.copy()
    v = v0.copy()
    n, m = u.shape

    eps = 1e-10
    h2inv = 4.0 ** (-level)    # 1/h^2
    gamma = reg * h2inv        # multiplies Laplacian neighbor contributions

    sweeps = max(1, s1 - int(level))

    # coefficients for local 2x2 blocks
    a = Ix * Ix + 4.0 * gamma
    b = Ix * Iy
    c = Iy * Iy + 4.0 * gamma
    det = a * c - b * b
    det[np.abs(det) < eps] = eps

    # Precompute coordinate parity mask
    I_idx = np.arange(n)
    J_idx = np.arange(m)
    par = (I_idx[:, None] + J_idx[None, :]) & 1  # 0/1 checkerboard

    for _ in range(sweeps):
        for p in (parity, 1 - parity):
            mask = (par == p)

            # neighbor sums (Dirichlet on boundary; we never update edges)
            Su = (u[:-2, 1:-1] + u[2:, 1:-1] +
                  u[1:-1, :-2] + u[1:-1, 2:])
            Sv = (v[:-2, 1:-1] + v[2:, 1:-1] +
                  v[1:-1, :-2] + v[1:-1, 2:])

            fu = rhsu[1:-1, 1:-1] + gamma * Su
            fv = rhsv[1:-1, 1:-1] + gamma * Sv

            a_loc = a[1:-1, 1:-1]
            b_loc = b[1:-1, 1:-1]
            c_loc = c[1:-1, 1:-1]
            det_loc = det[1:-1, 1:-1]

            u_new = (c_loc * fu - b_loc * fv) / det_loc
            v_new = (-b_loc * fu + a_loc * fv) / det_loc

            inner_mask = mask[1:-1, 1:-1]
            u[1:-1, 1:-1][inner_mask] = u_new[inner_mask]
            v[1:-1, 1:-1][inner_mask] = v_new[inner_mask]

        u = zero_boundary(u)
        v = zero_boundary(v)

    return u, v


def restriction(rhu, rhv, Ix, Iy):
    r2hu = restrict_simple(rhu)
    r2hv = restrict_simple(rhv)
    Ix2h = restrict_simple(Ix)
    Iy2h = restrict_simple(Iy)
    return r2hu, r2hv, Ix2h, Iy2h

def residual_MG(u, v, Ix, Iy, reg, rhsu, rhsv, level=0):
    """Residual r = b - A u for the multigrid operator."""
    Au, Av = A(u, v, Ix, Iy, reg, level=level)
    rhu = zero_boundary(rhsu.copy()) - Au
    rhv = zero_boundary(rhsv.copy()) - Av
    return rhu, rhv

def restrict_simple(rf):

    n_f, m_f = rf.shape
    n_c = n_f // 2
    m_c = m_f // 2

    r_c = np.zeros((n_c, m_c))

    # average each 2x2 block
    r_c[:, :] = 0.25 * (
        rf[0:n_f:2, 0:m_f:2] +
        rf[1:n_f:2, 0:m_f:2] +
        rf[0:n_f:2, 1:m_f:2] +
        rf[1:n_f:2, 1:m_f:2]
    )

    return r_c

def prolongation_MG(e2hu, e2hv):

    n_c, m_c = e2hu.shape
    n_f = 2 * (n_c - 1) + 1
    m_f = 2 * (m_c - 1) + 1

    ehu = np.zeros((n_f, m_f))
    ehv = np.zeros((n_f, m_f))

    # inject coarse nodes
    ehu[0::2, 0::2] = e2hu
    ehv[0::2, 0::2] = e2hv

    # interpolate vertical edges 
    ehu[1::2, 0::2] = 0.5 * (ehu[:-2:2, 0::2] + ehu[2::2, 0::2])
    ehv[1::2, 0::2] = 0.5 * (ehv[:-2:2, 0::2] + ehv[2::2, 0::2])

    # interpolate horizontal edges
    ehu[0::2, 1::2] = 0.5 * (ehu[0::2, :-2:2] + ehu[0::2, 2::2])
    ehv[0::2, 1::2] = 0.5 * (ehv[0::2, :-2:2] + ehv[0::2, 2::2])

    # interpolate centers
    ehu[1::2, 1::2] = 0.25 * (
        ehu[:-2:2, :-2:2] + ehu[:-2:2, 2::2] +
        ehu[2::2,  :-2:2] + ehu[2::2,  2::2]
    )
    ehv[1::2, 1::2] = 0.25 * (
        ehv[:-2:2, :-2:2] + ehv[:-2:2, 2::2] +
        ehv[2::2,  :-2:2] + ehv[2::2,  2::2]
    )

    return ehu, ehv


def OF_cg_level(u0, v0, Ix, Iy, reg, rhsu, rhsv,
                tol=1e-8, maxit=2000, level=0, verbose=False):
    """CG solver using the multigrid operator A on a given level."""
    u = zero_boundary(u0.copy())
    v = zero_boundary(v0.copy())

    Au, Av = A(u, v, Ix, Iy, reg, level)
    ru = zero_boundary(rhsu.copy()) - Au
    rv = zero_boundary(rhsv.copy()) - Av

    def dot2(a1, a2, b1, b2):
        return float(np.vdot(a1, b1) + np.vdot(a2, b2))

    r2_0 = dot2(ru, rv, ru, rv)
    if r2_0 == 0.0:
        return u, v, {"it": 0, "relres": 0.0, "converged": True}

    r0_norm = np.sqrt(r2_0)
    pu, pv = ru.copy(), rv.copy()
    relres = 1.0
    it = 0

    for k in range(1, maxit + 1):
        Apu, Apv = A(pu, pv, Ix, Iy, reg, level)
        denom = dot2(pu, pv, Apu, Apv)
        rr = dot2(ru, rv, ru, rv)
        alpha = rr / denom

        u += alpha * pu
        v += alpha * pv

        ru -= alpha * Apu
        rv -= alpha * Apv

        relres = np.sqrt(dot2(ru, rv, ru, rv)) / r0_norm
        if verbose and (k % 10 == 0 or relres < tol):
            print(f"[L{level}] CG it={k:3d} relres={relres:.3e}")
        if relres < tol:
            it = k
            break

        rr_new = dot2(ru, rv, ru, rv)
        beta = rr_new / rr
        pu = ru + beta * pu
        pv = rv + beta * pv
        it = k

    return u, v, {"it": it, "relres": relres, "converged": relres < tol}


def V_cycle(u0, v0, Ix, Iy, reg, rhsu, rhsv,
            s1, s2, level, max_level, parity=0):
    """
    Recursive V-cycle for the optical flow problem:
    - s1: number of pre-smoothing sweeps
    - s2: number of post-smoothing sweeps
    - level: current level (0 = finest)
    - max_level: total number of levels (e.g. 3 or 4)
    """
    # Pre-smoothing (RB order)
    u, v = smoothing(u0, v0, Ix, Iy, reg, rhsu, rhsv, s1, level=level, parity=0)

    # Compute residual
    rhu, rhv = residual_MG(u, v, Ix, Iy, reg, rhsu, rhsv, level=level)

    # Restrict to coarse grid
    r2hu, r2hv, Ix2h, Iy2h = restriction_MG(rhu, rhv, Ix, Iy)

    if level == max_level - 1:
        # Coarsest grid: solve approximately with CG
        eu0 = np.zeros_like(r2hu)
        ev0 = np.zeros_like(r2hv)
        eu, ev, _ = OF_cg_level(eu0, ev0, Ix2h, Iy2h, reg, r2hu, r2hv,
                                tol=1e-8, maxit=200, level=level+1,
                                verbose=False)
    else:
        # Recursive V-cycle on coarser grid
        eu0 = np.zeros_like(r2hu)
        ev0 = np.zeros_like(r2hv)
        eu, ev = V_cycle(eu0, ev0, Ix2h, Iy2h, reg, r2hu, r2hv,
                         s1, s2, level+1, max_level, parity=parity)

    # Prolongate error and correct
    ehu, ehv = prolongation_MG(eu, ev)
    u += ehu
    v += ehv

    # Post-smoothing with reversed parity (BR order)
    u, v = smoothing(u, v, Ix, Iy, reg, rhsu, rhsv, s2, level=level, parity=1)

    return u, v


def OF_pcg(u0, v0, Ix, Iy, reg, rhsu, rhsv,
           tol=1e-8, maxit=2000, level=0,
           s1=2, s2=2, max_level=3, verbose=False):
    """
    Preconditioned CG using a symmetric multigrid V-cycle as preconditioner.
    - s1, s2: pre- and post-smoothing steps (typically small, e.g. 1–3)
    - max_level: number of multigrid levels (0..max_level)
    """
    u = zero_boundary(u0.copy())
    v = zero_boundary(v0.copy())

    # Initial residual r0 = b - A u0
    Au, Av = A(u, v, Ix, Iy, reg, level)
    ru = zero_boundary(rhsu.copy()) - Au
    rv = zero_boundary(rhsv.copy()) - Av

    def dot2(a1, a2, b1, b2):
        return float(np.vdot(a1, b1) + np.vdot(a2, b2))

    r2_0 = dot2(ru, rv, ru, rv)
    if r2_0 == 0.0:
        return u, v, {"it": 0, "relres": 0.0, "converged": True, "res_hist": [0.0]}

    r0_norm = np.sqrt(r2_0)
    res_hist = [1.0]

    # z0 = M^{-1} r0 (apply V-cycle as preconditioner)
    zu, zv = V_cycle(np.zeros_like(ru), np.zeros_like(rv),
                     Ix, Iy, reg, ru, rv,
                     s1, s2, level, max_level, parity=0)

    pu = zu.copy()
    pv = zv.copy()

    rz_old = dot2(ru, rv, zu, zv)
    rel = 1.0
    it = 0

    while it < maxit and rel > tol:
        Apu, Apv = A(pu, pv, Ix, Iy, reg, level)
        denom = dot2(pu, pv, Apu, Apv)
        alpha = rz_old / denom

        # Update solution
        u += alpha * pu
        v += alpha * pv

        # Update residual
        ru -= alpha * Apu
        rv -= alpha * Apv

        r2_new = dot2(ru, rv, ru, rv)
        rel = np.sqrt(r2_new) / r0_norm
        res_hist.append(rel)

        if verbose and (it % 10 == 0 or rel <= tol):
            print(f"[PCG] it={it:4d} relres={rel:.3e}")

        if rel <= tol:
            break

        # z_{k+1} = M^{-1} r_{k+1} (apply V-cycle again)
        zu, zv = V_cycle(np.zeros_like(ru), np.zeros_like(rv),
                         Ix, Iy, reg, ru, rv,
                         s1, s2, level, max_level, parity=0)

        rz_new = dot2(ru, rv, zu, zv)
        beta = rz_new / rz_old

        pu = zu + beta * pu
        pv = zv + beta * pv

        rz_old = rz_new
        it += 1

    return u, v, {"it": it, "relres": rel, "converged": rel <= tol, "res_hist": res_hist}


def pad_to_odd(im):
    n, m = im.shape
    pad_n = 1 if n % 2 == 0 else 0
    pad_m = 1 if m % 2 == 0 else 0
    return np.pad(im, ((0, pad_n), (0, pad_m)), mode="edge")


def run_optical_flow(frame0_path, frame1_path,
                     lam=1.0, sigma=1.0,
                     tol=1e-8, maxit=2000):
    """Your original CG-based optical flow driver."""
    I0 = mpimg.imread(frame0_path).astype(float)
    I1 = mpimg.imread(frame1_path).astype(float)
    if I0.ndim == 3:
        I0 = np.mean(I0, axis=2)
        I1 = np.mean(I1, axis=2)

    I0 = gaussian_filter(I0, sigma)
    I1 = gaussian_filter(I1, sigma)

   


    Ix, Iy = compute_derivatives(I0, I1)
    It = I1 - I0

    rhsu, rhsv = build_right_hand_side(Ix, Iy, It)
    u0, v0 = np.zeros_like(I0), np.zeros_like(I0)

    u, v, info = OF_cg(u0, v0, Ix, Iy, lam, rhsu, rhsv, tol, maxit, verbose=True)

    print(f"CG finished in {info['it']} iterations, relres={info['relres']:.2e}, converged={info['converged']}")
    return u, v, info

def run_optical_flow_pcg(frame0_path, frame1_path,
                         lam=1.0, sigma=1.0,
                         tol=1e-8, maxit=2000,
                         s1=2, s2=2, max_level=3):
    """
    Optical flow driver using PCG with multigrid V-cycle preconditioner.
    """
    I0 = mpimg.imread(frame0_path).astype(float)
    I1 = mpimg.imread(frame1_path).astype(float)
    if I0.ndim == 3:
        I0 = np.mean(I0, axis=2)
        I1 = np.mean(I1, axis=2)

    I0 = gaussian_filter(I0, sigma)
    I1 = gaussian_filter(I1, sigma)

    I0 = pad_to_odd(I0)
    I1 = pad_to_odd(I1)

    Ix, Iy = compute_derivatives(I0, I1)
    It = I1 - I0

    rhsu, rhsv = build_right_hand_side(Ix, Iy, It)
    u0, v0 = np.zeros_like(I0), np.zeros_like(I0)

    u, v, info = OF_pcg(u0, v0, Ix, Iy, lam, rhsu, rhsv,
                        tol=tol, maxit=maxit,
                        s1=s1, s2=s2, max_level=max_level,
                        verbose=True)

    print(f"PCG+MG finished in {info['it']} iterations, "
          f"relres={info['relres']:.2e}, converged={info['converged']}")
    return u, v, info

frame0_path = "/Users/ethanclement/Documents/NLAProject/frame10.png"
frame1_path = "/Users/ethanclement/Documents/NLAProject/frame11.png"

u, v, info = run_optical_flow(frame0_path, frame1_path)
u_pcg, v_pcg, info_pcg = run_optical_flow_pcg(
    frame0_path,
    frame1_path,
    lam=1.0,
    sigma=1.0,
    tol=1e-8,
    maxit=2000,
    s1=2,
    s2=2,
    max_level=3,
)



it=  10 relres=1.436e+00
it=  20 relres=1.331e+00
it=  30 relres=1.210e+00
it=  40 relres=1.052e+00
it=  50 relres=8.778e-01
it=  60 relres=7.130e-01
it=  70 relres=5.967e-01
it=  80 relres=4.992e-01
it=  90 relres=4.154e-01
it= 100 relres=3.554e-01
it= 110 relres=3.082e-01
it= 120 relres=2.619e-01
it= 130 relres=2.096e-01
it= 140 relres=1.803e-01
it= 150 relres=1.507e-01
it= 160 relres=1.246e-01
it= 170 relres=9.923e-02
it= 180 relres=8.198e-02
it= 190 relres=7.041e-02
it= 200 relres=5.994e-02
it= 210 relres=5.189e-02
it= 220 relres=4.611e-02
it= 230 relres=3.973e-02
it= 240 relres=3.315e-02
it= 250 relres=2.804e-02
it= 260 relres=2.301e-02
it= 270 relres=1.887e-02
it= 280 relres=1.565e-02
it= 290 relres=1.289e-02
it= 300 relres=1.056e-02
it= 310 relres=8.685e-03
it= 320 relres=7.409e-03
it= 330 relres=6.280e-03
it= 340 relres=5.562e-03
it= 350 relres=4.698e-03
it= 360 relres=3.862e-03
it= 370 relres=3.240e-03
it= 380 relres=2.856e-03
it= 390 relres=2.458e-03
it= 400 relres=2.074e-03


ValueError: operands could not be broadcast together with shapes (241,321) (240,321) 